<a href="https://colab.research.google.com/github/KushangShah/Image-Classification-using-ML/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1


In [12]:
!ls -lah ~/datasets/dogs-vs-cats


total 20K
drwxr-xr-x 5 root root 4.0K Mar 14 09:28 .
drwxr-xr-x 3 root root 4.0K Mar 14 09:34 ..
drwxr-xr-x 4 root root 4.0K Mar 14 09:28 dogs_vs_cats
drwxr-xr-x 4 root root 4.0K Mar 14 09:28 test
drwxr-xr-x 4 root root 4.0K Mar 14 09:28 train


In [13]:
!mv ~/datasets/dogs-vs-cats /content/datasets/dogs-vs-cats

mv: cannot move '/root/datasets/dogs-vs-cats' to '/content/datasets/dogs-vs-cats': No such file or directory


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os